In [1]:
import pandas as pd

NROWS = None

title_vec = pd.read_csv('hn_title_norm_vec.csv', nrows=NROWS)

In [2]:
origin = pd.read_csv('/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv')
origin = origin[['id', 'num_points']]
data = pd.merge(title_vec, origin, how='inner', on=['id'])

In [3]:
'''
sentence_length = pd.read_csv('sentence_length.csv', header=None, )
sentence_length.rename(columns={0: 'id', 1:'length'}, inplace=True)
data = pd.merge(data, sentence_length, how='inner', on=['id'])
'''

"\nsentence_length = pd.read_csv('sentence_length.csv', header=None, )\nsentence_length.rename(columns={0: 'id', 1:'length'}, inplace=True)\ndata = pd.merge(data, sentence_length, how='inner', on=['id'])\n"

In [4]:
GOOD_THRESHOLD = 100

data['good'] = data['num_points'].apply(lambda x: 1 if x >= GOOD_THRESHOLD else 0)
print(data['good'].describe())

count    293119.000000
mean          0.039370
std           0.194473
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: good, dtype: float64


In [5]:
data = data.drop(['id', 'num_points'], 1)

In [6]:
train = data.sample(frac=0.8)
test = data.drop(train.index)

In [7]:
import numpy as np
good = train[train['good'] == 1]
bad = train[train['good'] == 0].head(good.shape[0])
trainer = good.append(bad)
train = trainer.reindex(np.random.permutation(trainer.index))

In [8]:
from keras.layers import Input, LSTM, RepeatVector, Dense, Activation, Bidirectional, Dropout, Convolution1D, GlobalMaxPooling1D, Flatten, MaxPooling1D
from keras.models import Model, Sequential

input_dim = 50
timesteps = 24
latent_dim = 200

predictor = Sequential()
#predictor.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(timesteps, input_dim), merge_mode='ave'))
#predictor.add(LSTM(100, input_shape=(timesteps, input_dim)))
predictor.add(Convolution1D(250, 3, activation='relu', input_shape=(timesteps, input_dim)))
predictor.add(GlobalMaxPooling1D())
#predictor.add(Flatten())
predictor.add(Dense(250))
predictor.add(Activation('relu'))

predictor.add(Dense(2))
predictor.add(Activation('softmax'))

'''
decoded = RepeatVector(timesteps)(encoded)
#decoded = LSTM(input_dim, return_sequences=True)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
'''

Using TensorFlow backend.


'\ndecoded = RepeatVector(timesteps)(encoded)\n#decoded = LSTM(input_dim, return_sequences=True)(encoded)\ndecoded = LSTM(input_dim, return_sequences=True)(decoded)\n\nsequence_autoencoder = Model(inputs, decoded)\nencoder = Model(inputs, encoded)\n'

In [9]:
from keras.utils.np_utils import to_categorical

label = ['good']

X = train.drop(label, axis=1)
Y = test.drop(label, axis=1)

X_label = to_categorical(train[label].as_matrix(), 2)
Y_label = to_categorical(test[label].as_matrix(), 2)

X = X.as_matrix().reshape(train.shape[0], 24, 50)
Y = Y.as_matrix().reshape(test.shape[0], 24, 50)

In [10]:
predictor.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'precision', 'recall', 'fmeasure'])
predictor.fit(X, X_label, validation_data=(Y, Y_label), batch_size=128)

Train on 18552 samples, validate on 58624 samples
Epoch 1/10
18552/18552 [==============================] - 14s - loss: 0.6825 - acc: 0.5581 - precision: 0.5581 - recall: 0.5581 - fmeasure: 0.5581 - val_loss: 0.6376 - val_acc: 0.6793 - val_precision: 0.6793 - val_recall: 0.6793 - val_fmeasure: 0.6793
Epoch 2/10
18552/18552 [==============================] - 14s - loss: 0.6602 - acc: 0.5991 - precision: 0.5991 - recall: 0.5991 - fmeasure: 0.5991 - val_loss: 0.7368 - val_acc: 0.4717 - val_precision: 0.4717 - val_recall: 0.4717 - val_fmeasure: 0.4717
Epoch 3/10
18552/18552 [==============================] - 14s - loss: 0.6303 - acc: 0.6420 - precision: 0.6420 - recall: 0.6420 - fmeasure: 0.6420 - val_loss: 0.6873 - val_acc: 0.5751 - val_precision: 0.5751 - val_recall: 0.5751 - val_fmeasure: 0.5751
Epoch 4/10
18552/18552 [==============================] - 14s - loss: 0.5917 - acc: 0.6792 - precision: 0.6792 - recall: 0.6792 - fmeasure: 0.6792 - val_loss: 0.6019 - val_acc: 0.6734 - val_prec

In [11]:
from keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=400)

In [12]:
y_train

array([1, 0, 0, ..., 0, 1, 0])